# Utility Function


In [7]:
import pandas as pd
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
import plotly.graph_objects as go
import math
import os
import gc
from math import sin, cos, sqrt, atan2, radians
from shapely.geometry import Point, Polygon

In [9]:
R = 6373.0 #earth radius
laneWidth=3
vehicleLength=5

In [10]:
column_names = ['CPM Count from Vehicles','Ego number', 'Source number','CPM Generated Time', 'CPM Received Time','Source Type','Time of Measurement','Ego ID', 'Ego speed', 'unit1','unit2', 'Ego Longitude','unit3', 'Ego Latitude','unit4', 'Ego Heading','unit5', 'CPM Object ID','Obejct Longitude', 'unit6','Obejct Latitude', 'unit7','Object speed', 'unit8','unit9', 'unit10','Object Heading', 'unit11',]
df = pd.read_csv('CPMstatsV2V.csv',sep=' ',names=column_names)

In [11]:
df['CPM delay'] = df['CPM Received Time']-df['CPM Generated Time']
df['Ego Heading in degrees'] = (df['Ego Heading']*180)/3.1415926535
df['longitude difference'] = df['Obejct Longitude']-df['Ego Longitude']
df['x'] = np.cos((df['Obejct Latitude']*3.1415926535)/180)*np.sin((df['longitude difference'])*3.1415926535/180)
df['y'] = (np.cos((df['Ego Latitude']*3.1415926535)/180)*np.sin((df['Obejct Latitude'])*3.1415926535/180))-(np.sin((df['Ego Latitude']*3.1415926535)/180)*np.cos((df['Obejct Latitude'])*3.1415926535/180)*np.cos((df['longitude difference'])*3.1415926535/180))
df['ego to object heading'] = np.arctan2(df['x'], df['y']) * 180 / np.pi
df['ego to object heading clockwise']=df['ego to object heading']

df['Ego Longitude in radian'] = np.radians(df['Ego Longitude'])
df['Ego Latitude in radian'] = np.radians(df['Ego Latitude'])
df['Obejct Longitude in radian'] = np.radians(df['Obejct Longitude'])
df['Obejct Latitude in radian'] = np.radians(df['Obejct Latitude'])

df['Ego to Object Distance'] = df['Ego Longitude']

df['Ego to Object Distance'] = np.sin((df['Obejct Latitude in radian']-df['Ego Latitude in radian']) / 2)**2 + np.cos(df['Ego Latitude in radian']) * np.cos(df['Obejct Latitude in radian']) * np.sin((df['Obejct Longitude in radian']-df['Ego Longitude in radian']) / 2)**2

df['Ego to Object Distance'] =   1000 * R * (2 * np.arctan2(np.sqrt(df['Ego to Object Distance']), np.sqrt(1 - df['Ego to Object Distance'])))



In [12]:
df.sort_values(by=['Ego number','CPM Object ID','CPM Received Time'], inplace=True,ascending = [True, True, True])


In [ ]:
df.reset_index(inplace=True)

In [14]:
df.drop(columns=['index'],inplace=True)

In [8]:
grouped = df.groupby('Ego number')

In [9]:
output_folder = '/home/ramesh/Downloads/ITSC/results/lust_120_results/10/'


In [10]:
groupname=[]

In [11]:
for name, group in grouped:
    filename = os.path.join(output_folder, f"group_{name}.csv")
    groupname.append(f"group_{name}.csv")
    group.to_csv(filename, index=False)

In [12]:
gc.collect()

26

# Utility Basic Calculation

In [13]:

angles=[30,60,90,120,150,180,210,240,270,300,330,359.99]
g=0

for group in groupname:
    file_path = os.path.join(output_folder, group)
    df = pd.read_csv(file_path)
    print(g)
    g=g+1
    

    for angle in angles:
        df[f"useful_{angle}"]=df['ego to object heading']
        df[f"usefulesss_{angle}"]=df['ego to object heading']
        df['object angle thresh1']=df['CPM delay']
        df['object angle thresh2']=df['CPM delay']

        df['object angle thresh1 clockwise']=df['CPM delay']
        df['object angle thresh2 clockwise']=df['CPM delay']
        for i in range (0,len(df)):
            if int(df['ego to object heading'][i]) <0:
                df['ego to object heading clockwise'][i]=360+df['ego to object heading'][i]
            else:
                df['ego to object heading clockwise'][i]=df['ego to object heading'][i]
            df[f"useful_{angle}"][i]=0
            df[f"usefulesss_{angle}"][i]=0
            df['object angle thresh1'][i]=df['Ego Heading in degrees'][i]-(angle/2)
            if df['Ego Heading in degrees'][i]+(angle/2)>360:
                df['object angle thresh2'][i]=df['Ego Heading in degrees'][i]+(angle/2)-360
            else:
                df['object angle thresh2'][i]=df['Ego Heading in degrees'][i]+(angle/2)
            if int(df['object angle thresh1'][i]) <0:
                df['object angle thresh1 clockwise'][i]=360+df['object angle thresh1'][i]
            else:
                df['object angle thresh1 clockwise'][i]=df['object angle thresh1'][i]
            if int(df['object angle thresh2'][i]) <0:
                df['object angle thresh2 clockwise'][i]=360+df['object angle thresh2'][i]
            else:
                df['object angle thresh2 clockwise'][i]=df['object angle thresh2'][i] 

            if float(df['object angle thresh1 clockwise'][i])<float(df['object angle thresh2 clockwise'][i]):
                if float(df['object angle thresh1 clockwise'][i])<float(df['ego to object heading clockwise'][i])<float(df['object angle thresh2 clockwise'][i]):
                    df[f"useful_{angle}"][i]=1
                else:
                     df[f"usefulesss_{angle}"][i]=1 

            elif float(df['object angle thresh2 clockwise'][i])<float(df['object angle thresh1 clockwise'][i]):
                if float(df['object angle thresh2 clockwise'][i])<float(df['ego to object heading clockwise'][i])<float(df['object angle thresh1 clockwise'][i]):

                    df[f"usefulesss_{angle}"][i]=1 
                else:
                    df[f"useful_{angle}"][i]=1
            else:
                print('something wrong')
                
                

    filename = os.path.join(output_folder, group)            
    df.to_csv(filename, index=False)
    gc.collect()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [14]:
for angle in angles:
    print(angle)
    needed=0
    noneeded=0
    for group in groupname:
        file_path = os.path.join(output_folder, group)
        df = pd.read_csv(file_path)
        for i in range(0,len(df)):
            needed=needed+float(df[f"useful_{angle}"][i])
            noneeded=noneeded+float(df[f"usefulesss_{angle}"][i])
    gc.collect()
    print(needed)
    print(noneeded)
    print(needed+noneeded)
    print("")

30
146964.0
452032.0
598996.0

60
184620.0
414376.0
598996.0

90
208461.0
390535.0
598996.0

120
230048.0
368948.0
598996.0

150
254152.0
344844.0
598996.0

180
284261.0
314735.0
598996.0

210
305383.0
293613.0
598996.0

240
324306.0
274690.0
598996.0

270
345086.0
253910.0
598996.0

300
376024.0
222972.0
598996.0

330
435490.0
163506.0
598996.0

359.99
598927.0
69.0
598996.0



In [15]:
distances=[25,50,75,100,125,150,175,200,225,250]

for distance in distances: 
    print(distance)
    needed=0
    noneeded=0
    for group in groupname:
        file_path = os.path.join(output_folder, group)
        df = pd.read_csv(file_path)
        for i in range (0,len(df)):
            if float(df['Ego to Object Distance'][i]) <=distance:
                needed=needed+1
            else:
                noneeded=noneeded+1
    gc.collect()            
    print(needed)
    print(noneeded)
    print(needed+noneeded)
    print("")

25
25347
573649
598996

50
53335
545661
598996

75
77480
521516
598996

100
101798
497198
598996

125
119875
479121
598996

150
135300
463696
598996

175
150665
448331
598996

200
166340
432656
598996

225
185959
413037
598996

250
206559
392437
598996



# Hybrid Utility Function

In [16]:
detection_threshold_range=250

for angle in angles:
    print(angle)
    needed=0
    noneeded=0
    
    for group in groupname:
        file_path = os.path.join(output_folder, group)
        df = pd.read_csv(file_path)
        df[f"useful hybrid{angle}"]=df['ego to object heading']
        df[f"usefulesss hybrid{angle}"]=df['ego to object heading']
        
        for i in range(0,len(df)):
            if float(df['Ego to Object Distance'][i]) <=detection_threshold_range and float(df[f"useful_{angle}"][i])==1.0:
                needed=needed+1
                df[f"useful hybrid{angle}"][i]=1
                df[f"usefulesss hybrid{angle}"][i]=0
            else:
                noneeded=noneeded+1
                df[f"useful hybrid{angle}"][i]=0
                df[f"usefulesss hybrid{angle}"][i]=1
        
        filename = os.path.join(output_folder, group)            
        df.to_csv(filename, index=False)
        gc.collect()

    print(needed)
    print(noneeded)
    print(needed+noneeded)
    print("")


30
62606
536390
598996

60
73085
525911
598996

90
80808
518188
598996

120
87246
511750
598996

150
93984
505012
598996

180
100742
498254
598996

210
104877
494119
598996

240
109088
489908
598996

270
114040
484956
598996

300
122820
476176
598996

330
139477
459519
598996

359.99
206517
392479
598996



In [17]:
column_names = ['CPM Count from Vehicles','Ego number', 'Source number','CPM Generated Time', 'CPM Received Time','Source Type','Time of Measurement','Ego ID', 'Ego speed', 'unit1','unit2', 'Ego Longitude','unit3', 'Ego Latitude','unit4', 'Ego Heading','unit5', 'CPM Object ID','Obejct Longitude', 'unit6','Obejct Latitude', 'unit7','Object speed', 'unit8','unit9', 'unit10','Object Heading', 'unit11',]
dfs = pd.read_csv('CPMstatsV2V.csv',sep=' ',names=column_names)

In [18]:
dfs.sort_values(by=['Ego number','CPM Object ID','CPM Received Time'], inplace=True,ascending = [True, True, True])
dfs.reset_index(inplace=True)
dfs.drop(columns=['index'],inplace=True)

In [19]:
groupsforfinal = dfs.groupby('Source number')

# Half cycle danger zone

In [20]:
for angle in angles:
    print(angle)
    enter_danger_zone_value=0
    no_enter_danger_zone_value=0
    useful=0
    value=0
    total=0
    identified=0
    unidentified=0
    CAM=0
    mylist=[]
    camlist=[]
    
    
    for group in groupname:
        file_path = os.path.join(output_folder, group)
        df = pd.read_csv(file_path)
        df.sort_values(by=['Ego number','CPM Object ID','CPM Received Time'], inplace=True,ascending = [True, True, True])
        df.reset_index(inplace=True)
        df.drop(columns=['index'],inplace=True)
        #print(df)
    
    
        for i in range (0,len(df)-1):
            if  df[f"usefulesss hybrid{angle}"][i]==1.0 and df['Ego number'][i]==df['Ego number'][i+1] and df['CPM Object ID'][i]==df['CPM Object ID'][i+1]:
                if -90<=df['ego to object heading'][i+1]<=90 and df['Ego to Object Distance'][i+1]<df['Ego speed'][i]*3:
                    enter_danger_zone_value=enter_danger_zone_value+1
                    ego_number=df['Ego number'][i]
                    object_id=df['CPM Object ID'][i]
                    received_time=df['CPM Received Time'][i+1]
                    cam_not_detected= True
                    if object_id in dfs['Ego ID'].unique():
                        CAM=CAM+1
                        cam_not_detected= False
                    if cam_not_detected: 
                        value=value+1
                        tempdf = groupsforfinal.get_group(ego_number)
                        tempdf = tempdf.reset_index()
                        tempdf.drop(columns=['index'],inplace=True)
                        for l in range (0,len(tempdf)-1):
                            if tempdf['CPM Object ID'][l]==object_id and (float(received_time) - float(tempdf['CPM Generated Time'][l])) >= 0.0 and (float(received_time) - float(tempdf['CPM Generated Time'][l]))<= 1.0:
                                identified=identified+1
                                break
                else:
                    no_enter_danger_zone_value=no_enter_danger_zone_value+1
            else:
                useful=useful+1
        gc.collect()
    print("Results hybrid")        
    print("enter danger zone value", enter_danger_zone_value)
    print("no enter danger zone value", no_enter_danger_zone_value)
    print("useful", useful)

    print("value", value)
    print("CAM", CAM)
    print("identified by radar", identified)
    print("identified by CAM + radar", CAM + identified)
    print("")

30
Results hybrid
enter danger zone value 14405
no enter danger zone value 506369
useful 77664
value 3784
CAM 10621
identified by radar 685
identified by CAM + radar 11306

60
Results hybrid
enter danger zone value 13201
no enter danger zone value 497276
useful 87961
value 3368
CAM 9833
identified by radar 297
identified by CAM + radar 10130

90
Results hybrid
enter danger zone value 12492
no enter danger zone value 490389
useful 95557
value 3172
CAM 9320
identified by radar 162
identified by CAM + radar 9482

120
Results hybrid
enter danger zone value 12023
no enter danger zone value 484513
useful 101902
value 3052
CAM 8971
identified by radar 107
identified by CAM + radar 9078

150
Results hybrid
enter danger zone value 11496
no enter danger zone value 478392
useful 108550
value 2903
CAM 8593
identified by radar 71
identified by CAM + radar 8664

180
Results hybrid
enter danger zone value 11190
no enter danger zone value 472042
useful 115206
value 2827
CAM 8363
identified by radar 60

# Rectangle danger zone

In [22]:
for angle in angles:
    print(angle)


    enter_danger_zone_value=0
    no_enter_danger_zone_value=0
    useful=0
    value=0
    total=0
    identified=0
    unidentified=0
    CAM=0
    mylist=[]
    camlist=[]
    
    
    for group in groupname:
        file_path = os.path.join(output_folder, group)
        df = pd.read_csv(file_path)
        df.sort_values(by=['Ego number','CPM Object ID','CPM Received Time'], inplace=True,ascending = [True, True, True])
        df.reset_index(inplace=True)
        df.drop(columns=['index'],inplace=True)
    
    
        for i in range (0,len(df)-1):
            if  df[f"usefulesss hybrid{angle}"][i]==1.0 and df['Ego number'][i]==df['Ego number'][i+1] and df['CPM Object ID'][i]==df['CPM Object ID'][i+1]:
        
                # Initial point (source) coordinates in degrees
                egolat = math.radians(df['Ego Latitude'][i])
                egolon = math.radians(df['Ego Longitude'][i])
                egoheading = (df['Ego Heading in degrees'][i])
                
                x1=math.sqrt((df['Ego speed'][i]*3)**2+(laneWidth/2)**2)
                x2=math.sqrt((vehicleLength/2)**2+(laneWidth/2)**2)
                
                x1Angle=math.degrees(math.atan((laneWidth/2)/((df['Ego speed'][i]*3))))
                x2Angle=math.degrees(math.atan((laneWidth/2)/(vehicleLength)))
                

                # Bearing angle in degrees
                if egoheading - x1Angle < 0:
                    angle1 = egoheading - x1Angle + 360
                elif egoheading - x1Angle > 360:
                    angle1 = egoheading - x1Angle - 360
                else:
                    angle1 = egoheading - x1Angle

                if egoheading + x1Angle < 0:
                    angle2 = egoheading + x1Angle + 360
                elif egoheading + x1Angle > 360:
                    angle2 = egoheading + x1Angle - 360
                else:
                    angle2 = egoheading + x1Angle

                if egoheading -180 - x2Angle < 0:
                    angle3 = egoheading -180 - x2Angle + 360
                elif egoheading -180 - x2Angle > 360:
                    angle3 = egoheading -180 - x2Angle - 360
                else:
                    angle3 = egoheading -180 - x2Angle

                if egoheading -180 + x2Angle < 0:
                    angle4 = egoheading -180 + x2Angle + 360
                elif egoheading -180 + x2Angle > 360:
                    angle4 = egoheading -180 + x2Angle - 360
                else:
                    angle4 = egoheading -180 + x2Angle

                theta1 = math.radians(angle1)
                theta2 = math.radians(angle2)
                theta3 = math.radians(angle3)
                theta4 = math.radians(angle4)

                # Distance in kilometers
                d1 = x1/1000 #convert to Km
                d2 = x1/1000
                d3 = x2/1000
                d4 = x2/1000

                # Calculate the destination point (latitude and longitude)
                lat1 = math.asin(math.sin(egolat) * math.cos(d1 / R) + math.cos(egolat) * math.sin(d1 / R) * math.cos(theta1))
                lon1 = egolon + math.atan2(math.sin(theta1) * math.sin(d1 / R) * math.cos(egolat), math.cos(d1 / R) - math.sin(egolat) * math.sin(egolat))

                lat2 = math.asin(math.sin(egolat) * math.cos(d2 / R) + math.cos(egolat) * math.sin(d2 / R) * math.cos(theta2))
                lon2 = egolon + math.atan2(math.sin(theta2) * math.sin(d2 / R) * math.cos(egolat), math.cos(d2 / R) - math.sin(egolat) * math.sin(egolat))

                lat3 = math.asin(math.sin(egolat) * math.cos(d3 / R) + math.cos(egolat) * math.sin(d3 / R) * math.cos(theta3))
                lon3 = egolon + math.atan2(math.sin(theta3) * math.sin(d3 / R) * math.cos(egolat), math.cos(d3 / R) - math.sin(egolat) * math.sin(egolat))

                lat4 = math.asin(math.sin(egolat) * math.cos(d4 / R) + math.cos(egolat) * math.sin(d4 / R) * math.cos(theta4))
                lon4 = egolon + math.atan2(math.sin(theta4) * math.sin(d4 / R) * math.cos(egolat), math.cos(d4 / R) - math.sin(egolat) * math.sin(egolat))

                # Convert the result back to degrees
                destination_latitude1 = math.degrees(lat1)
                destination_longitude1 = math.degrees(lon1)

                destination_latitude2 = math.degrees(lat2)
                destination_longitude2 = math.degrees(lon2)

                destination_latitude3 = math.degrees(lat3)
                destination_longitude3 = math.degrees(lon3)

                destination_latitude4 = math.degrees(lat4)
                destination_longitude4 = math.degrees(lon4)

                coordinates = [(destination_longitude1, destination_latitude1), (destination_longitude2, destination_latitude2), (destination_longitude3, destination_latitude3), (destination_longitude4, destination_latitude4)]

                polygon = Polygon(coordinates)

                # Define the test point
                test_point = Point(df['Obejct Longitude'][i+1], df['Obejct Latitude'][i+1])  # Replace with your test coordinates

                # Check if the test point is within the polygon
                is_inside = test_point.within(polygon)


                if is_inside:
                    enter_danger_zone_value=enter_danger_zone_value+1
                    ego_number=df['Ego number'][i]
                    object_id=df['CPM Object ID'][i]
                    received_time=df['CPM Received Time'][i+1]
                    
                    cam_not_detected= True
                    if object_id in dfs['Ego ID'].unique():
                        CAM=CAM+1
                        cam_not_detected= False
                    if cam_not_detected: 
                        value=value+1
                        tempdf = groupsforfinal.get_group(ego_number)
                        tempdf = tempdf.reset_index()
                        tempdf.drop(columns=['index'],inplace=True)
                        for l in range (0,len(tempdf)-1):
                            if tempdf['CPM Object ID'][l]==object_id and (float(received_time) - float(tempdf['CPM Generated Time'][l])) >= 0.0 and (float(received_time) - float(tempdf['CPM Generated Time'][l]))<= 1.0:
                                identified=identified+1
                                break
                else:
                    no_enter_danger_zone_value=no_enter_danger_zone_value+1
            else:
                useful=useful+1
        gc.collect()
    print("Results hybrid")        
    print("enter danger zone value", enter_danger_zone_value)
    print("no enter danger zone value", no_enter_danger_zone_value)
    print("useful", useful)

    print("value", value)
    print("CAM", CAM)
    print("identified by radar", identified)
    print("identified by CAM + radar", CAM + identified)
    print("")


30


/home/ramesh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning:

divide by zero encountered in double_scalars



Results hybrid
enter danger zone value 1130
no enter danger zone value 519644
useful 77664
value 378
CAM 752
identified by radar 67
identified by CAM + radar 819

60
Results hybrid
enter danger zone value 1060
no enter danger zone value 509417
useful 87961
value 355
CAM 705
identified by radar 45
identified by CAM + radar 750

90
Results hybrid
enter danger zone value 1019
no enter danger zone value 501862
useful 95557
value 340
CAM 679
identified by radar 33
identified by CAM + radar 712

120
Results hybrid
enter danger zone value 987
no enter danger zone value 495549
useful 101902
value 327
CAM 660
identified by radar 21
identified by CAM + radar 681

150
Results hybrid
enter danger zone value 968
no enter danger zone value 488920
useful 108550
value 323
CAM 645
identified by radar 20
identified by CAM + radar 665

180
Results hybrid
enter danger zone value 953
no enter danger zone value 482279
useful 115206
value 314
CAM 639
identified by radar 18
identified by CAM + radar 657

210


# Dynamic Utility Function

# Linear Function

In [23]:
g=0
m= (180/27.77)
c=180
angleList=[]

num_groups = groupsforfinal.ngroups
print(num_groups)
for group in groupname:
    file_path = os.path.join(output_folder, group)
    df = pd.read_csv(file_path)
    print(g)
    g=g+1
    
    
    df['useful_velocity_linear_location']=df['ego to object heading']
    df['usefulesss_velocity_linear_location']=df['ego to object heading']
    df['object angle thresh1']=df['CPM delay']
    df['object angle thresh2']=df['CPM delay']
    
    df['angle with linear and location']=df['CPM delay']

    df['object angle thresh1 clockwise']=df['CPM delay']
    df['object angle thresh2 clockwise']=df['CPM delay']
    for i in range (0,len(df)):
        ego_number=df['Ego number'][i]
        if ego_number in groupsforfinal.groups:
            tempdf = groupsforfinal.get_group(ego_number)
            tempdf = tempdf.reset_index()
            tempdf.drop(columns=['index'],inplace=True)
            received_time=df['CPM Received Time'][i]
            angle= m*df['Ego speed'][i] + c
            if angle>=360:
                angle=359.99
            for l in range (0,len(tempdf)-1):
                if (float(received_time) - float(tempdf['CPM Generated Time'][l])) >= 0.0 and (float(received_time) - float(tempdf['CPM Generated Time'][l]))<= 1.0:
                    ego_long=df['Ego Longitude'][i]
                    ego_lat=df['Ego Latitude'][i]
                    object_long=tempdf['Obejct Longitude'][l]
                    object_lat=tempdf['Obejct Latitude'][l]
                
                    longitude_difference= object_long - ego_long
                    x= np.cos((object_lat*3.1415926535)/180)*np.sin((longitude_difference)*3.1415926535/180)
                    y = (np.cos((ego_lat*3.1415926535)/180)*np.sin((object_lat)*3.1415926535/180))-(np.sin((ego_lat*3.1415926535)/180)*np.cos((object_lat)*3.1415926535/180)*np.cos((longitude_difference)*3.1415926535/180))
                    ego_to_object_heading= np.arctan2(x, y) * 180 / np.pi
                                
                    R = 6373.0
                    Ego_Longitude_in_radian = np.radians(ego_long)
                    Ego_Latitude_in_radian = np.radians(ego_lat)
                    Obejct_Longitude_in_radian = np.radians(object_long)
                    Obejct_Latitude_in_radian = np.radians(object_lat)


                    Ego_to_Object_Distance = np.sin((Obejct_Latitude_in_radian - Ego_Latitude_in_radian) / 2)**2 + np.cos(Ego_Latitude_in_radian) * np.cos(Obejct_Latitude_in_radian) * np.sin((Obejct_Longitude_in_radian - Ego_Longitude_in_radian) / 2)**2

                    Ego_to_Object_Distance =   1000 * R * (2 * np.arctan2(np.sqrt(Ego_to_Object_Distance), np.sqrt(1 - Ego_to_Object_Distance)))
                    if Ego_to_Object_Distance>10 and 45<=abs(ego_to_object_heading)<=135:
                        angle=359.99
                        break
        
                #    break
        else:
            angle= m*df['Ego speed'][i] + c
            if angle>=360:
                angle=359.99
            
        

        angleList.append(angle)
        df['angle with linear and location'][i]=angle
        if int(df['ego to object heading'][i]) <0:
            df['ego to object heading clockwise'][i]=360+df['ego to object heading'][i]
        else:
            df['ego to object heading clockwise'][i]=df['ego to object heading'][i]
        df['useful_velocity_linear_location'][i]=0
        df['usefulesss_velocity_linear_location'][i]=0
        df['object angle thresh1'][i]=df['Ego Heading in degrees'][i]-(angle/2)
        if df['Ego Heading in degrees'][i]+(angle/2)>360:
            df['object angle thresh2'][i]=df['Ego Heading in degrees'][i]+(angle/2)-360
        else:
            df['object angle thresh2'][i]=df['Ego Heading in degrees'][i]+(angle/2)
        if int(df['object angle thresh1'][i]) <0:
            df['object angle thresh1 clockwise'][i]=360+df['object angle thresh1'][i]
        else:
            df['object angle thresh1 clockwise'][i]=df['object angle thresh1'][i]
        if int(df['object angle thresh2'][i]) <0:
            df['object angle thresh2 clockwise'][i]=360+df['object angle thresh2'][i]
        else:
            df['object angle thresh2 clockwise'][i]=df['object angle thresh2'][i] 

        if float(df['object angle thresh1 clockwise'][i])<float(df['object angle thresh2 clockwise'][i]):
            if float(df['object angle thresh1 clockwise'][i])<float(df['ego to object heading clockwise'][i])<float(df['object angle thresh2 clockwise'][i]):
                df['useful_velocity_linear_location'][i]=1
            else:
                df['usefulesss_velocity_linear_location'][i]=1 

        elif float(df['object angle thresh2 clockwise'][i])<float(df['object angle thresh1 clockwise'][i]):
            if float(df['object angle thresh2 clockwise'][i])<float(df['ego to object heading clockwise'][i])<float(df['object angle thresh1 clockwise'][i]):

                df['usefulesss_velocity_linear_location'][i]=1 
            else:
                df['useful_velocity_linear_location'][i]=1
        else:
            print('something wrong')
                
                

    filename = os.path.join(output_folder, group)            
    df.to_csv(filename, index=False)
    gc.collect()

6.481814908174289
442
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271


In [86]:
detection_threshold_range=100
needed=0
noneeded=0
    
for group in groupname:
    file_path = os.path.join(output_folder, group)
    df = pd.read_csv(file_path)
    df['useful_velocity_linear_location_hybrid']=df['ego to object heading']
    df['usefulesss_velocity_linear_location_hybrid']=df['ego to object heading']
        
    for i in range(0,len(df)):
        if float(df['Ego to Object Distance'][i]) <=detection_threshold_range and float(df['useful_velocity_linear_location'][i])==1.0:
            needed=needed+1
            df['useful_velocity_linear_location_hybrid'][i]=1
            df['usefulesss_velocity_linear_location_hybrid'][i]=0
        else:
            noneeded=noneeded+1
            df['useful_velocity_linear_location_hybrid'][i]=0
            df['usefulesss_velocity_linear_location_hybrid'][i]=1
        
    filename = os.path.join(output_folder, group)            
    df.to_csv(filename, index=False)
    gc.collect()

print(needed)
print(noneeded)
print(needed+noneeded)
print("")


77747
521249
598996



# Half cycle danger zone

In [87]:
enter_danger_zone_value=0
no_enter_danger_zone_value=0
useful=0
value=0
total=0
identified=0
unidentified=0
CAM=0
mylist=[]
camlist=[]


for group in groupname:
    file_path = os.path.join(output_folder, group)
    df = pd.read_csv(file_path)
    df.sort_values(by=['Ego number','CPM Object ID','CPM Received Time'], inplace=True,ascending = [True, True, True])
    df.reset_index(inplace=True)
    df.drop(columns=['index'],inplace=True)


    for i in range (0,len(df)-1):
        if  df['usefulesss_velocity_linear_location_hybrid'][i]==1.0 and df['Ego number'][i]==df['Ego number'][i+1] and df['CPM Object ID'][i]==df['CPM Object ID'][i+1]:
            if -90<=df['ego to object heading'][i+1]<=90 and df['Ego to Object Distance'][i+1]<df['Ego speed'][i]*3:
                enter_danger_zone_value=enter_danger_zone_value+1
                ego_number=df['Ego number'][i]
                object_id=df['CPM Object ID'][i]
                received_time=df['CPM Received Time'][i+1]
                cam_not_detected= True
                if object_id in dfs['Ego ID'].unique():
                    CAM=CAM+1
                    cam_not_detected= False
                if cam_not_detected: 
                    value=value+1
                    tempdf = groupsforfinal.get_group(ego_number)
                    tempdf = tempdf.reset_index()
                    tempdf.drop(columns=['index'],inplace=True)
                    for l in range (0,len(tempdf)-1):
                        if tempdf['CPM Object ID'][l]==object_id and (float(received_time) - float(tempdf['CPM Generated Time'][l])) >= 0.0 and (float(received_time) - float(tempdf['CPM Generated Time'][l]))<= 1.0:
                            identified=identified+1
                            break
            else:
                no_enter_danger_zone_value=no_enter_danger_zone_value+1
        else:
            useful=useful+1
    gc.collect()
print("Results hybrid")        
print("enter danger zone value", enter_danger_zone_value)
print("no enter danger zone value", no_enter_danger_zone_value)
print("useful", useful)

print("value", value)
print("CAM", CAM)
print("identified by radar", identified)
print("identified by CAM + radar", CAM + identified)
print("")


Results hybrid
enter danger zone value 3445
no enter danger zone value 502217
useful 92776
value 767
CAM 2678
identified by radar 23
identified by CAM + radar 2701



# Rectangle danger zone

In [88]:
enter_danger_zone_value=0
no_enter_danger_zone_value=0
useful=0
value=0
total=0
identified=0
unidentified=0
CAM=0
mylist=[]
camlist=[]


for group in groupname:
    file_path = os.path.join(output_folder, group)
    df = pd.read_csv(file_path)
    df.sort_values(by=['Ego number','CPM Object ID','CPM Received Time'], inplace=True,ascending = [True, True, True])
    df.reset_index(inplace=True)
    df.drop(columns=['index'],inplace=True)


    for i in range (0,len(df)-1):
        if  df['usefulesss_velocity_linear_location_hybrid'][i]==1.0 and df['Ego number'][i]==df['Ego number'][i+1] and df['CPM Object ID'][i]==df['CPM Object ID'][i+1]:

            # Initial point (source) coordinates in degrees
            egolat = math.radians(df['Ego Latitude'][i])
            egolon = math.radians(df['Ego Longitude'][i])
            egoheading = (df['Ego Heading in degrees'][i])

            x1=math.sqrt((df['Ego speed'][i]*3)**2+(laneWidth/2)**2)
            x2=math.sqrt((vehicleLength/2)**2+(laneWidth/2)**2)

            x1Angle=math.degrees(math.atan((laneWidth/2)/((df['Ego speed'][i]*3))))
            x2Angle=math.degrees(math.atan((laneWidth/2)/(vehicleLength)))


            # Bearing angle in degrees
            if egoheading - x1Angle < 0:
                angle1 = egoheading - x1Angle + 360
            elif egoheading - x1Angle > 360:
                angle1 = egoheading - x1Angle - 360
            else:
                angle1 = egoheading - x1Angle

            if egoheading + x1Angle < 0:
                angle2 = egoheading + x1Angle + 360
            elif egoheading + x1Angle > 360:
                angle2 = egoheading + x1Angle - 360
            else:
                angle2 = egoheading + x1Angle

            if egoheading -180 - x2Angle < 0:
                angle3 = egoheading -180 - x2Angle + 360
            elif egoheading -180 - x2Angle > 360:
                angle3 = egoheading -180 - x2Angle - 360
            else:
                angle3 = egoheading -180 - x2Angle

            if egoheading -180 + x2Angle < 0:
                angle4 = egoheading -180 + x2Angle + 360
            elif egoheading -180 + x2Angle > 360:
                angle4 = egoheading -180 + x2Angle - 360
            else:
                angle4 = egoheading -180 + x2Angle



            theta1 = math.radians(angle1)
            theta2 = math.radians(angle2)
            theta3 = math.radians(angle3)
            theta4 = math.radians(angle4)

            # Distance in kilometers
            d1 = x1/1000 #convert to Km
            d2 = x1/1000
            d3 = x2/1000
            d4 = x2/1000

            # Calculate the destination point (latitude and longitude)
            lat1 = math.asin(math.sin(egolat) * math.cos(d1 / R) + math.cos(egolat) * math.sin(d1 / R) * math.cos(theta1))
            lon1 = egolon + math.atan2(math.sin(theta1) * math.sin(d1 / R) * math.cos(egolat), math.cos(d1 / R) - math.sin(egolat) * math.sin(egolat))

            lat2 = math.asin(math.sin(egolat) * math.cos(d2 / R) + math.cos(egolat) * math.sin(d2 / R) * math.cos(theta2))
            lon2 = egolon + math.atan2(math.sin(theta2) * math.sin(d2 / R) * math.cos(egolat), math.cos(d2 / R) - math.sin(egolat) * math.sin(egolat))

            lat3 = math.asin(math.sin(egolat) * math.cos(d3 / R) + math.cos(egolat) * math.sin(d3 / R) * math.cos(theta3))
            lon3 = egolon + math.atan2(math.sin(theta3) * math.sin(d3 / R) * math.cos(egolat), math.cos(d3 / R) - math.sin(egolat) * math.sin(egolat))

            lat4 = math.asin(math.sin(egolat) * math.cos(d4 / R) + math.cos(egolat) * math.sin(d4 / R) * math.cos(theta4))
            lon4 = egolon + math.atan2(math.sin(theta4) * math.sin(d4 / R) * math.cos(egolat), math.cos(d4 / R) - math.sin(egolat) * math.sin(egolat))

            # Convert the result back to degrees
            destination_latitude1 = math.degrees(lat1)
            destination_longitude1 = math.degrees(lon1)

            destination_latitude2 = math.degrees(lat2)
            destination_longitude2 = math.degrees(lon2)

            destination_latitude3 = math.degrees(lat3)
            destination_longitude3 = math.degrees(lon3)

            destination_latitude4 = math.degrees(lat4)
            destination_longitude4 = math.degrees(lon4)

            coordinates = [(destination_longitude1, destination_latitude1), (destination_longitude2, destination_latitude2), (destination_longitude3, destination_latitude3), (destination_longitude4, destination_latitude4)]

            polygon = Polygon(coordinates)

            # Define the test point
            test_point = Point(df['Obejct Longitude'][i+1], df['Obejct Latitude'][i+1])  # Replace with your test coordinates

            # Check if the test point is within the polygon
            is_inside = test_point.within(polygon)


            if is_inside:
                enter_danger_zone_value=enter_danger_zone_value+1
                ego_number=df['Ego number'][i]
                object_id=df['CPM Object ID'][i]
                received_time=df['CPM Received Time'][i+1]

                cam_not_detected= True
                if object_id in dfs['Ego ID'].unique():
                    CAM=CAM+1
                    cam_not_detected= False
                if cam_not_detected: 
                    value=value+1
                    tempdf = groupsforfinal.get_group(ego_number)
                    tempdf = tempdf.reset_index()
                    tempdf.drop(columns=['index'],inplace=True)
                    for l in range (0,len(tempdf)-1):
                        if tempdf['CPM Object ID'][l]==object_id and (float(received_time) - float(tempdf['CPM Generated Time'][l])) >= 0.0 and (float(received_time) - float(tempdf['CPM Generated Time'][l]))<= 1.0:
                            identified=identified+1
                            break
            else:
                no_enter_danger_zone_value=no_enter_danger_zone_value+1
        else:
            useful=useful+1
    gc.collect()
print("Results hybrid")        
print("enter danger zone value", enter_danger_zone_value)
print("no enter danger zone value", no_enter_danger_zone_value)
print("useful", useful)

print("value", value)
print("CAM", CAM)
print("identified by radar", identified)
print("identified by CAM + radar", CAM + identified)
print("")


/home/ramesh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning:

divide by zero encountered in double_scalars



Results hybrid
enter danger zone value 386
no enter danger zone value 505276
useful 92776
value 124
CAM 262
identified by radar 5
identified by CAM + radar 267



# Exponential Utility Function

In [37]:
g=0
a=1
b=0.1869
c=180
#print(m)
angleList=[]

num_groups = groupsforfinal.ngroups
print(num_groups)
for group in groupname:
    file_path = os.path.join(output_folder, group)
    df = pd.read_csv(file_path)
    print(g)
    g=g+1
    
    df['useful_velocity_exp_location']=df['ego to object heading']
    df['usefulesss_velocity_exp_location']=df['ego to object heading']
    df['object angle thresh1']=df['CPM delay']
    df['object angle thresh2']=df['CPM delay']
    
    df['angle with exp and location']=df['CPM delay']

    df['object angle thresh1 clockwise']=df['CPM delay']
    df['object angle thresh2 clockwise']=df['CPM delay']
    for i in range (0,len(df)):
        ego_number=df['Ego number'][i]
        if ego_number in groupsforfinal.groups:
            tempdf = groupsforfinal.get_group(ego_number)
            tempdf = tempdf.reset_index()
            tempdf.drop(columns=['index'],inplace=True)
            received_time=df['CPM Received Time'][i]
            angle= a*np.exp(df['Ego speed'][i]*b) + c
            if angle>=360:
                angle=359.99
            for l in range (0,len(tempdf)-1):
                if (float(received_time) - float(tempdf['CPM Generated Time'][l])) >= 0.0 and (float(received_time) - float(tempdf['CPM Generated Time'][l]))<= 1.0:
                    ego_long=df['Ego Longitude'][i]
                    ego_lat=df['Ego Latitude'][i]
                    object_long=tempdf['Obejct Longitude'][l]
                    object_lat=tempdf['Obejct Latitude'][l]
                
                    longitude_difference= object_long - ego_long
                    x= np.cos((object_lat*3.1415926535)/180)*np.sin((longitude_difference)*3.1415926535/180)
                    y = (np.cos((ego_lat*3.1415926535)/180)*np.sin((object_lat)*3.1415926535/180))-(np.sin((ego_lat*3.1415926535)/180)*np.cos((object_lat)*3.1415926535/180)*np.cos((longitude_difference)*3.1415926535/180))
                    ego_to_object_heading= np.arctan2(x, y) * 180 / np.pi
                                
                    R = 6373.0
                    Ego_Longitude_in_radian = np.radians(ego_long)
                    Ego_Latitude_in_radian = np.radians(ego_lat)
                    Obejct_Longitude_in_radian = np.radians(object_long)
                    Obejct_Latitude_in_radian = np.radians(object_lat)


                    Ego_to_Object_Distance = np.sin((Obejct_Latitude_in_radian - Ego_Latitude_in_radian) / 2)**2 + np.cos(Ego_Latitude_in_radian) * np.cos(Obejct_Latitude_in_radian) * np.sin((Obejct_Longitude_in_radian - Ego_Longitude_in_radian) / 2)**2

                    Ego_to_Object_Distance =   1000 * R * (2 * np.arctan2(np.sqrt(Ego_to_Object_Distance), np.sqrt(1 - Ego_to_Object_Distance)))
                    if Ego_to_Object_Distance>10 and 45<=abs(ego_to_object_heading)<=135:
                        angle=359.99
                        break
        
                #    break
        else:
            angle= a*np.exp(df['Ego speed'][i]*b) + c
            if angle>=360:
                angle=359.99
            
        
        angleList.append(angle)
        df['angle with exp and location'][i]=angle
        if int(df['ego to object heading'][i]) <0:
            df['ego to object heading clockwise'][i]=360+df['ego to object heading'][i]
        else:
            df['ego to object heading clockwise'][i]=df['ego to object heading'][i]
        df['useful_velocity_exp_location'][i]=0
        df['usefulesss_velocity_exp_location'][i]=0
        df['object angle thresh1'][i]=df['Ego Heading in degrees'][i]-(angle/2)
        if df['Ego Heading in degrees'][i]+(angle/2)>360:
            df['object angle thresh2'][i]=df['Ego Heading in degrees'][i]+(angle/2)-360
        else:
            df['object angle thresh2'][i]=df['Ego Heading in degrees'][i]+(angle/2)
        if int(df['object angle thresh1'][i]) <0:
            df['object angle thresh1 clockwise'][i]=360+df['object angle thresh1'][i]
        else:
            df['object angle thresh1 clockwise'][i]=df['object angle thresh1'][i]
        if int(df['object angle thresh2'][i]) <0:
            df['object angle thresh2 clockwise'][i]=360+df['object angle thresh2'][i]
        else:
            df['object angle thresh2 clockwise'][i]=df['object angle thresh2'][i] 

        if float(df['object angle thresh1 clockwise'][i])<float(df['object angle thresh2 clockwise'][i]):
            if float(df['object angle thresh1 clockwise'][i])<float(df['ego to object heading clockwise'][i])<float(df['object angle thresh2 clockwise'][i]):
                df['useful_velocity_exp_location'][i]=1
            else:
                df['usefulesss_velocity_exp_location'][i]=1 

        elif float(df['object angle thresh2 clockwise'][i])<float(df['object angle thresh1 clockwise'][i]):
            if float(df['object angle thresh2 clockwise'][i])<float(df['ego to object heading clockwise'][i])<float(df['object angle thresh1 clockwise'][i]):

                df['usefulesss_velocity_exp_location'][i]=1 
            else:
                df['useful_velocity_exp_location'][i]=1
        else:
            print('something wrong')
                
                

    filename = os.path.join(output_folder, group)            
    df.to_csv(filename, index=False)
    gc.collect()

442
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
27

In [89]:
detection_threshold_range=100
needed=0
noneeded=0
    
for group in groupname:
    file_path = os.path.join(output_folder, group)
    df = pd.read_csv(file_path)
    df['useful_velocity_exp_location_hybrid']=df['ego to object heading']
    df['usefulesss_velocity_exp_location_hybrid']=df['ego to object heading']
        
    for i in range(0,len(df)):
        if float(df['Ego to Object Distance'][i]) <=detection_threshold_range and float(df['useful_velocity_exp_location'][i])==1.0:
            needed=needed+1
            df['useful_velocity_exp_location_hybrid'][i]=1
            df['usefulesss_velocity_exp_location_hybrid'][i]=0
        else:
            noneeded=noneeded+1
            df['useful_velocity_exp_location_hybrid'][i]=0
            df['usefulesss_velocity_exp_location_hybrid'][i]=1
        
    filename = os.path.join(output_folder, group)            
    df.to_csv(filename, index=False)
    gc.collect()

print(needed)
print(noneeded)
print(needed+noneeded)
print("")

75946
523050
598996



# Half cycle danger zone

In [90]:
enter_danger_zone_value=0
no_enter_danger_zone_value=0
useful=0
value=0
total=0
identified=0
unidentified=0
CAM=0
mylist=[]
camlist=[]


for group in groupname:
    file_path = os.path.join(output_folder, group)
    df = pd.read_csv(file_path)
    df.sort_values(by=['Ego number','CPM Object ID','CPM Received Time'], inplace=True,ascending = [True, True, True])
    df.reset_index(inplace=True)
    df.drop(columns=['index'],inplace=True)


    for i in range (0,len(df)-1):
        if  df['usefulesss_velocity_exp_location_hybrid'][i]==1.0 and df['Ego number'][i]==df['Ego number'][i+1] and df['CPM Object ID'][i]==df['CPM Object ID'][i+1]:
            if -90<=df['ego to object heading'][i+1]<=90 and df['Ego to Object Distance'][i+1]<df['Ego speed'][i]*3:
                enter_danger_zone_value=enter_danger_zone_value+1
                ego_number=df['Ego number'][i]
                object_id=df['CPM Object ID'][i]
                received_time=df['CPM Received Time'][i+1]
                cam_not_detected= True
                if object_id in dfs['Ego ID'].unique():
                    CAM=CAM+1
                    cam_not_detected= False
                if cam_not_detected: 
                    value=value+1
                    tempdf = groupsforfinal.get_group(ego_number)
                    tempdf = tempdf.reset_index()
                    tempdf.drop(columns=['index'],inplace=True)
                    for l in range (0,len(tempdf)-1):
                        if tempdf['CPM Object ID'][l]==object_id and (float(received_time) - float(tempdf['CPM Generated Time'][l])) >= 0.0 and (float(received_time) - float(tempdf['CPM Generated Time'][l]))<= 1.0:
                            identified=identified+1
                            break
            else:
                no_enter_danger_zone_value=no_enter_danger_zone_value+1
        else:
            useful=useful+1
    gc.collect()
print("Results hybrid")        
print("enter danger zone value", enter_danger_zone_value)
print("no enter danger zone value", no_enter_danger_zone_value)
print("useful", useful)

print("value", value)
print("CAM", CAM)
print("identified by radar", identified)
print("identified by CAM + radar", CAM + identified)
print("")
#print(unidentified)
#print(CAM+identified)

#print(total)
#print(total/value)

Results hybrid
enter danger zone value 4054
no enter danger zone value 503384
useful 91000
value 934
CAM 3120
identified by radar 33
identified by CAM + radar 3153



# Rectangle danger zone

In [91]:
enter_danger_zone_value=0
no_enter_danger_zone_value=0
useful=0
value=0
total=0
identified=0
unidentified=0
CAM=0
mylist=[]
camlist=[]


for group in groupname:
    file_path = os.path.join(output_folder, group)
    df = pd.read_csv(file_path)
    df.sort_values(by=['Ego number','CPM Object ID','CPM Received Time'], inplace=True,ascending = [True, True, True])
    df.reset_index(inplace=True)
    df.drop(columns=['index'],inplace=True)


    for i in range (0,len(df)-1):
        if  df['usefulesss_velocity_exp_location_hybrid'][i]==1.0 and df['Ego number'][i]==df['Ego number'][i+1] and df['CPM Object ID'][i]==df['CPM Object ID'][i+1]:

            # Initial point (source) coordinates in degrees
            egolat = math.radians(df['Ego Latitude'][i])
            egolon = math.radians(df['Ego Longitude'][i])
            egoheading = (df['Ego Heading in degrees'][i])

            x1=math.sqrt((df['Ego speed'][i]*3)**2+(laneWidth/2)**2)
            x2=math.sqrt((vehicleLength/2)**2+(laneWidth/2)**2)

            x1Angle=math.degrees(math.atan((laneWidth/2)/((df['Ego speed'][i]*3))))
            x2Angle=math.degrees(math.atan((laneWidth/2)/(vehicleLength)))


            # Bearing angle in degrees
            if egoheading - x1Angle < 0:
                angle1 = egoheading - x1Angle + 360
            elif egoheading - x1Angle > 360:
                angle1 = egoheading - x1Angle - 360
            else:
                angle1 = egoheading - x1Angle

            if egoheading + x1Angle < 0:
                angle2 = egoheading + x1Angle + 360
            elif egoheading + x1Angle > 360:
                angle2 = egoheading + x1Angle - 360
            else:
                angle2 = egoheading + x1Angle

            if egoheading -180 - x2Angle < 0:
                angle3 = egoheading -180 - x2Angle + 360
            elif egoheading -180 - x2Angle > 360:
                angle3 = egoheading -180 - x2Angle - 360
            else:
                angle3 = egoheading -180 - x2Angle

            if egoheading -180 + x2Angle < 0:
                angle4 = egoheading -180 + x2Angle + 360
            elif egoheading -180 + x2Angle > 360:
                angle4 = egoheading -180 + x2Angle - 360
            else:
                angle4 = egoheading -180 + x2Angle


            theta1 = math.radians(angle1)
            theta2 = math.radians(angle2)
            theta3 = math.radians(angle3)
            theta4 = math.radians(angle4)

            # Distance in kilometers
            d1 = x1/1000 #convert to Km
            d2 = x1/1000
            d3 = x2/1000
            d4 = x2/1000

            # Calculate the destination point (latitude and longitude)
            lat1 = math.asin(math.sin(egolat) * math.cos(d1 / R) + math.cos(egolat) * math.sin(d1 / R) * math.cos(theta1))
            lon1 = egolon + math.atan2(math.sin(theta1) * math.sin(d1 / R) * math.cos(egolat), math.cos(d1 / R) - math.sin(egolat) * math.sin(egolat))

            lat2 = math.asin(math.sin(egolat) * math.cos(d2 / R) + math.cos(egolat) * math.sin(d2 / R) * math.cos(theta2))
            lon2 = egolon + math.atan2(math.sin(theta2) * math.sin(d2 / R) * math.cos(egolat), math.cos(d2 / R) - math.sin(egolat) * math.sin(egolat))

            lat3 = math.asin(math.sin(egolat) * math.cos(d3 / R) + math.cos(egolat) * math.sin(d3 / R) * math.cos(theta3))
            lon3 = egolon + math.atan2(math.sin(theta3) * math.sin(d3 / R) * math.cos(egolat), math.cos(d3 / R) - math.sin(egolat) * math.sin(egolat))

            lat4 = math.asin(math.sin(egolat) * math.cos(d4 / R) + math.cos(egolat) * math.sin(d4 / R) * math.cos(theta4))
            lon4 = egolon + math.atan2(math.sin(theta4) * math.sin(d4 / R) * math.cos(egolat), math.cos(d4 / R) - math.sin(egolat) * math.sin(egolat))

            # Convert the result back to degrees
            destination_latitude1 = math.degrees(lat1)
            destination_longitude1 = math.degrees(lon1)

            destination_latitude2 = math.degrees(lat2)
            destination_longitude2 = math.degrees(lon2)

            destination_latitude3 = math.degrees(lat3)
            destination_longitude3 = math.degrees(lon3)

            destination_latitude4 = math.degrees(lat4)
            destination_longitude4 = math.degrees(lon4)

            coordinates = [(destination_longitude1, destination_latitude1), (destination_longitude2, destination_latitude2), (destination_longitude3, destination_latitude3), (destination_longitude4, destination_latitude4)]

            polygon = Polygon(coordinates)

            # Define the test point
            test_point = Point(df['Obejct Longitude'][i+1], df['Obejct Latitude'][i+1])  # Replace with your test coordinates

            # Check if the test point is within the polygon
            is_inside = test_point.within(polygon)


            if is_inside:
                enter_danger_zone_value=enter_danger_zone_value+1
                ego_number=df['Ego number'][i]
                object_id=df['CPM Object ID'][i]
                received_time=df['CPM Received Time'][i+1]

                cam_not_detected= True
                if object_id in dfs['Ego ID'].unique():
                    CAM=CAM+1
                    cam_not_detected= False
                if cam_not_detected: 
                    value=value+1
                    tempdf = groupsforfinal.get_group(ego_number)
                    tempdf = tempdf.reset_index()
                    tempdf.drop(columns=['index'],inplace=True)
                    for l in range (0,len(tempdf)-1):
                        if tempdf['CPM Object ID'][l]==object_id and (float(received_time) - float(tempdf['CPM Generated Time'][l])) >= 0.0 and (float(received_time) - float(tempdf['CPM Generated Time'][l]))<= 1.0:
                            identified=identified+1
                            break
            else:
                no_enter_danger_zone_value=no_enter_danger_zone_value+1
        else:
            useful=useful+1
    gc.collect()
print("Results hybrid")        
print("enter danger zone value", enter_danger_zone_value)
print("no enter danger zone value", no_enter_danger_zone_value)
print("useful", useful)

print("value", value)
print("CAM", CAM)
print("identified by radar", identified)
print("identified by CAM + radar", CAM + identified)
print("")

/home/ramesh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning:

divide by zero encountered in double_scalars



Results hybrid
enter danger zone value 452
no enter danger zone value 506986
useful 91000
value 141
CAM 311
identified by radar 9
identified by CAM + radar 320

